<a href="https://colab.research.google.com/github/Templasan/DSM---Projeto-de-API-1-Semestre/blob/main/Graficos/Pesquisa_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Carregar e tratar os dados**

In [2]:
#Bibliotecas Necessarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

#Dados de 2013
exportacoes2013 = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_2013_MUN.csv'

#Tabelas para tradução dos codigos
mun = 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
sh4 = 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
pais = 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'

#Leitura da tabela 2013 e tabelas de codigos
df = pd.read_csv(exportacoes2013, sep=";", encoding="latin1")
df_mun = pd.read_csv(mun, sep=";", encoding="latin1", usecols=['CO_MUN_GEO', 'NO_MUN'])
df_sh4 = pd.read_csv(sh4, sep=';', encoding='latin-1', usecols=['CO_SH4', 'NO_SH4_POR'])
df_pais = pd.read_csv(pais, sep=";",encoding="latin1", usecols=['CO_PAIS', 'NO_PAIS'])

#Renomear colunas antes de mesclar dados para compatibilidade
df_mun = df_mun.rename(columns={"CO_MUN_GEO": "CO_MUN"})
df_sh4 = df_sh4.rename(columns={"CO_SH4": "SH4", "NO_SH4_POR": "PRODUTO"})

#Remover duplicatas de sh4
df_sh4 = df_sh4.drop_duplicates(subset=["SH4"])

#Mesclar DF
df_mescla = df.merge(df_mun, on=["CO_MUN"], how="left")
df_mescla = df_mescla.merge(df_sh4, on=["SH4"], how="left")
df_mescla = df_mescla.merge(df_pais, on=["CO_PAIS"], how="left")

#Criando coluna valor agregado
df_mescla["VALOR_AGREGADO"] = df_mescla["VL_FOB"] / df_mescla["KG_LIQUIDO"]

#Tratar casos onde KG_LIQUIDO é zero para evitar divisão por zero
df_mescla = df_mescla[df_mescla["KG_LIQUIDO"] > 0].reset_index()
display(df_mescla.head())

#Renomear Colunas
df_mescla = df_mescla.rename(columns ={'CO_ANO': 'ANO', 'CO_MES': 'MES', 'SG_UF_MUN': 'UF', 'NO_MUN': 'MUN', 'NO_PAIS': 'PAIS' })

#Filtrar UF para SP
df_mescla = df_mescla.loc[df_mescla['UF'] == 'SP']

,index,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_MUN,PRODUTO,NO_PAIS,VALOR_AGREGADO
0,0,2013,6,8501,63,PR,4106902,780,76547,CURITIBA,"Motores e geradores, elétricos, exceto os grup...",Argentina,98.137179
1,1,2013,6,9503,586,SC,4213203,886,9277,POMERODE,Outros brinquedos; modelos reduzidos e modelos...,Paraguai,10.470655
2,2,2013,6,207,403,SC,4208203,1216585,2266251,ITAJAI,"Carnes e miudezas comestíveis, frescas, refrig...",Jordânia,1.862797
3,3,2013,6,8511,158,SP,3438709,166,4060,PIRACICABA,Aparelhos e dispositivos elétricos de ignição ...,Chile,24.457831
4,4,2013,6,8409,97,RS,4305108,46,1637,CAXIAS DO SUL,Partes reconhecíveis como exclusiva ou princip...,Bolívia,35.586957


# **Pesquisa de Informações por Municípios**

In [3]:
# Escolher o município de interesse (exemplo: 'SAO PAULO')
municipio_especifico = input("Digite o nome do município: ")

df_municipio = df_mescla[df_mescla["MUN"].str.upper() == municipio_especifico.upper()]

# Corrected line: using a list of column names within double square brackets
display(df_municipio[['PRODUTO', 'PAIS', 'VALOR_AGREGADO']].sample(10))

Digite o nome do município: sao paulo


,PRODUTO,PAIS,VALOR_AGREGADO
394866,Partes e acessórios dos veículos automóveis da...,Honduras,8.436302
358298,"Banheiras, « chuveiros », pias, lavatórios, bi...",Angola,15.491228
81662,"Motores de pistão, de ignição por compressão (...",Chile,15.852997
72878,"Celulose e seus derivados químicos, não especi...",Chile,3.650114
403895,"Camisolas e pulôveres, cardigans, coletes e ar...",Japão,156.000000
306292,"Outras chapas, folhas, películas, tiras e lâmi...",Angola,69.333333
627292,Uvas frescas ou secas,Canadá,3.216856
497475,Máquinas automáticas para processamento de dad...,Estados Unidos,1199.691176
178045,Outros tubos e perfis ocos (por exemplo: solda...,Congo,5.972051
549819,Outras obras de plástico e obras de outras mat...,Tailândia,31.182390


# **Pesquisa de Informação por Código**

In [17]:
# Escolher o município de interesse (exemplo: 'SAO PAULO')
codigo_sh4 = input("Digite o código SH4: ")

df_produto = df_mescla[df_mescla["SH4"] == int(codigo_sh4)]

# Agrupar por município e somar o valor FOB
df_exportacao_mun = df_produto.groupby(['MUN'])['VL_FOB'].sum().sort_values(ascending=False)

# Obter o nome do produto correspondente ao código SH4
nome_produto = df_sh4.loc[df_sh4['SH4'] == int(codigo_sh4), 'PRODUTO'].iloc[0]

print(f"\n Os municípios que mais exportam o produto \n {nome_produto})\n são:")
display(df_exportacao_mun)

Digite o código SH4: 8512

 Os municípios que mais exportam o produto 
 Aparelhos elétricos de iluminação ou de sinalização (exceto os da posição 8539), limpadores de pára-brisas, degeladores e desembaciadores elétricos, dos tipos utilizados em ciclos e automóveis)
 são:


,VL_FOB
MUN,
SAO BERNARDO DO CAMPO,56229579
SAO CAETANO DO SUL,10401342
SAO PAULO,9060725
VINHEDO,7131022
CAMPINAS,5272851
GUARULHOS,5163867
SOROCABA,3770379
DIADEMA,3366145
JUNDIAI,3291389
